In [25]:
#age in code ro nafahmidin khodetoon ye code joda ba estefade az StreamData_preprocessed.csv benevisin. yani
#ye code benevisin ke StreamData_preprocessed.csv ro load kone va satr hasho yeki yeki pass bede be codetoon.
#input: 7-column-rows
#output: send the rows to elasticsearch and show it.
import json,datetime
from kafka import KafkaConsumer
import pandas as pd
from elasticsearch import Elasticsearch

topicToReceive = 'elasticsearch'
kafkaPort = 19092
ElasticsearchPort = 9200
elasticIndex = 'bigdatafinalproject3'
First_Time = True

In [3]:
def current_time():
    current_datetime = datetime.datetime.now()
    return current_datetime.strftime("[%Y-%m-%d_%H:%M:%S]")

consumer = KafkaConsumer(topicToReceive, bootstrap_servers=[f'127.0.0.1:{kafkaPort}'])

In [7]:
es = Elasticsearch(hosts=['http://localhost:9200'])
print(f'{current_time()}: Connected to elasticsearch')

[2023-07-14_17:15:00]: Connected to elasticsearch


In [26]:
if First_Time:
    index_name = elasticIndex
    index_body = {
        'settings': {
            'number_of_shards': 1,
            'number_of_replicas': 0
        },
        'mappings': {
            'properties': {
                    "date": {
                        "type": "date"
                    },
                    "message": {
                        "type": "text"
                    },
                    "hashtags": {
                        "type": "keyword"
                    },
                    "score": {
                        "type": "integer"
                    },
                    "category": {
                        "type": "keyword"
                    },
                    "clean_message": {
                        "type": "text"
                    },
                    "ML_score": {
                        "type": "float"
                    }
                
            }
        }
    }
    es.indices.create(index=index_name, body=index_body)
    print(f"{current_time()}: First_time is on. created the index called {elasticIndex}")
    First_Time = False


C:\Users\AhmaDGoly\AppData\Local\Temp\ipykernel_9000\1792738971.py:35: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index=index_name, body=index_body)


[2023-07-14_17:30:21]: First_time is on. created the index called bigdatafinalproject3


In [19]:
import dateutil.parser

In [29]:
x = 0
for message in consumer:
    x = x + 1
    print(f"{current_time()}: {x}: Received a news from Preprocessing_Thread...")
    json_row = message.value.decode('utf-8')
    row = json.loads(json_row)
    receivedRowDF = pd.DataFrame([row])
    #az inja be bad kodetoon ro be ezaye har satre jadidi ke omad piade konin.
    receivedRowDF['date'] = receivedRowDF['date'].apply(lambda x: dateutil.parser.parse(x).isoformat())
    records = receivedRowDF.to_dict(orient='records')
    for record in records:
        res = es.index(index=elasticIndex, document=record)
    print(f"{current_time()}: {x}: Saved on Elasticsearch on index = {elasticIndex}")

[2023-07-14_17:32:11]: 1: Received a news from Preprocessing_Thread...
[2023-07-14_17:32:11]: 1: Saved on Elasticsearch on index = bigdatafinalproject3
[2023-07-14_17:32:31]: 2: Received a news from Preprocessing_Thread...
[2023-07-14_17:32:32]: 2: Saved on Elasticsearch on index = bigdatafinalproject3
[2023-07-14_17:32:52]: 3: Received a news from Preprocessing_Thread...
[2023-07-14_17:32:52]: 3: Saved on Elasticsearch on index = bigdatafinalproject3
[2023-07-14_17:33:16]: 4: Received a news from Preprocessing_Thread...
[2023-07-14_17:33:16]: 4: Saved on Elasticsearch on index = bigdatafinalproject3
[2023-07-14_17:33:34]: 5: Received a news from Preprocessing_Thread...
[2023-07-14_17:33:34]: 5: Saved on Elasticsearch on index = bigdatafinalproject3
[2023-07-14_17:33:55]: 6: Received a news from Preprocessing_Thread...
[2023-07-14_17:33:55]: 6: Saved on Elasticsearch on index = bigdatafinalproject3
[2023-07-14_17:34:12]: 7: Received a news from Preprocessing_Thread...
[2023-07-14_17:34

ConnectionTimeout: Connection timed out